## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Torres,BR,-29.3353,-49.7269,68.92,85,34,4.32,scattered clouds
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,73.24,91,100,3.65,overcast clouds
2,2,Castro,BR,-24.7911,-50.0119,61.09,98,90,6.76,overcast clouds
3,3,San Patricio,US,28.0170,-97.5169,61.00,77,90,10.36,overcast clouds
4,4,Georgetown,MY,5.4112,100.3354,89.01,70,20,6.91,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Georgetown,MY,5.4112,100.3354,89.01,70,20,6.91,few clouds
6,6,Atuona,PF,-9.8000,-139.0333,79.75,76,48,12.93,scattered clouds
7,7,Palu,ID,-0.8917,119.8707,85.32,67,97,2.89,overcast clouds
10,10,Bathsheba,BB,13.2167,-59.5167,78.80,65,20,21.85,few clouds
14,14,Carnarvon,AU,-24.8667,113.6333,84.20,84,0,16.11,clear sky
17,17,Faanui,PF,-16.4833,-151.7500,81.91,79,100,18.41,light rain
18,18,Vaini,TO,-21.2000,-175.2000,80.60,83,75,13.80,light rain
20,20,Saint-Philippe,RE,-21.3585,55.7679,84.20,70,31,5.75,scattered clouds
21,21,Avarua,CK,-21.2078,-159.7750,78.80,94,75,17.27,light rain
24,24,Kawalu,ID,-7.3817,108.2082,84.85,66,74,1.14,light rain


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                192
City                   192
Country                192
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [34]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Georgetown,MY,89.01,few clouds,5.4112,100.3354,
6,Atuona,PF,79.75,scattered clouds,-9.8000,-139.0333,
7,Palu,ID,85.32,overcast clouds,-0.8917,119.8707,
10,Bathsheba,BB,78.80,few clouds,13.2167,-59.5167,
14,Carnarvon,AU,84.20,clear sky,-24.8667,113.6333,
17,Faanui,PF,81.91,light rain,-16.4833,-151.7500,
18,Vaini,TO,80.60,light rain,-21.2000,-175.2000,
20,Saint-Philippe,RE,84.20,scattered clouds,-21.3585,55.7679,
21,Avarua,CK,78.80,light rain,-21.2078,-159.7750,
24,Kawalu,ID,84.85,light rain,-7.3817,108.2082,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", float("NaN"), inplace=True)
clean_hotel_df = hotel_df.dropna()

In [53]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [55]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))